# TensorFlow script mode training and serving

This makes use of nima and transfer learning from mobilenet to make image classification

In [45]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

# Convert AVA dataset into TFRecord

Convert AVA dataset into TFRecord

Here is the entire script:

In [38]:
#!/usr/bin/env python3
"""Script for converting AVA dataset."""

import random
import tensorflow as tf

# The directory where the AVA dataset is stored
ava_dir = '/home/ec2-user/ava_dataset'

# The directory where the TFRecord files will be stored
dataset_dir = '/home/ec2-user/ava_dataset/tfrecords'

# Number of examples per TFRecord shard
shard_num = 10000

# Fraction of dataset for validation
validation_split = 0.2


with open(os.path.join(ava_dir, 'AVA.txt'), 'r') as f:
    ava = [line.strip().split() for line in f.readlines()]

image_path = tf.placeholder(dtype=tf.string)
jpeg = tf.read_file(image_path)
decoded = tf.image.decode_jpeg(jpeg, channels=3)

counts = {'train': 0, 'validation': 0}
writers = {}

#os.makedirs(dataset_dir)

with tf.Session() as sess:
    for item in ava:
        filename = os.path.join(ava_dir, 'photos', item[1]) + '.jpg'
        try:
            image_data, _ = sess.run(
                [jpeg, decoded], feed_dict={image_path: filename})

            if random.random() > validation_split:
                split = 'train'
            else:
                split = 'validation'

            if split not in writers or counts[split] % shard_num == 0:
                writer_path = os.path.join(
                    dataset_dir, '{}_{}-{}.tfrecord'.format(
                        split, counts[split],
                        counts[split] + shard_num - 1))
                writers[split] = tf.python_io.TFRecordWriter(writer_path)

            scores = tf.train.FloatList(value=list(map(int, item[2:12])))
            image = tf.train.BytesList(value=[image_data])
            features = tf.train.Features(feature={
                'scores': tf.train.Feature(float_list=scores),
                'image': tf.train.Feature(bytes_list=image)})
            example = tf.train.Example(features=features)
            writers[split].write(example.SerializeToString())
            counts[split] += 1
        except:
            print('Error decoding image: {}'.format(filename))

for split, count in counts.items():
    filename = '{}.txt'.format(split)
    with open(os.path.join(dataset_dir, filename), 'w') as f:
        f.write('{}\n'.format(count))




# Construct a script for distributed training



In [124]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [144]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm.py',
                       source_dir='nima',
                       role=role,
                       framework_version='1.12.0',
                       py_version='py3',
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, train_instance_type='ml.m5.xlarge')

In [ ]:
estimator.fit({'train' :  's3://waynetoh-ml/ava_dataset/train',
             'validation' :  's3://waynetoh-ml/ava_dataset/train',
             'eval' :  's3://waynetoh-ml/ava_dataset/train'})

# Deploy the trained model to an endpoint

The `deploy()` method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint, because we trained with script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol. The [Using your own inference code]() document explains how SageMaker runs inference containers.

# Invoke the endpoint

Let's download the training data and use that as input for inference.

# Delete the endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.